In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
trash = pd.read_csv('trash_hauler_report.csv')
trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,11/01/17,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,11/01/17,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


In [3]:
trash.columns = trash.columns.str.strip()

In [4]:
missed_categories = ['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup']
missed_categories

['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup']

In [5]:
# red_river_missed = trash[(trash['Trash Hauler'] == 'RED RIVER') & (trash['Request'].isin(missed_categories))].copy()
# red_river_missed.head()

In [6]:
# red_river_missed['Incident Address'].isna().sum()

In [7]:
# red_river_missed = red_river_missed.dropna(subset = ['Incident Address'])
# red_river_missed

In [8]:
### remove  (trash['Description'].str.contains(pattern, case=False, na=False)) and there are 12911 rows

pattern = r"\bmiss\w*\b|not\s+emptied|not\s+picked\s+up|no\s+pick\s+up" #miss/missed, not emptied, not picked up

if isinstance(missed_categories, list):
    missed_pattern = '|'.join(missed_categories)
else:
    missed_pattern = missed_categories 

rrmissed = trash[
    (trash['Trash Hauler'].str.contains("Red River", case=False, na=False)) &
    (
        (trash['Request'].str.contains(missed_pattern, case=False, na=False)) |
        (trash['Description'].str.contains(pattern, case=False, na=False))
    )
]

### drop known missing address

rrmissed = rrmissed.dropna(subset = ['Incident Address'])

rrmissed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,11/01/17,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066
8,25330,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1.794534e+06,618749.342732
...,...,...,...,...,...,...,...,...,...,...,...
20220,267121,11/01/19,Trash - Curbside/Alley Missed Pickup,missed,"2709 Crestdale Dr, Nashville, TN 37214, United...",37214.0,RED RIVER,1502,15.0,1.770240e+06,676334.399319
20221,267125,11/01/19,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1.781137e+06,632448.551144
20223,267130,11/01/19,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1.770293e+06,674936.303809
20224,267134,11/01/19,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1.785225e+06,627146.400187


In [9]:
rrmissed = rrmissed.drop_duplicates(subset=['Incident Address', 'Date Opened'])
rrmissed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,11/01/17,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066
8,25330,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1.794534e+06,618749.342732
...,...,...,...,...,...,...,...,...,...,...,...
20218,267110,11/01/19,Trash - Backdoor,Friday/paid backdoor trash,3105 fisher dr,37214.0,RED RIVER,1503,15.0,1.767923e+06,673264.801310
20220,267121,11/01/19,Trash - Curbside/Alley Missed Pickup,missed,"2709 Crestdale Dr, Nashville, TN 37214, United...",37214.0,RED RIVER,1502,15.0,1.770240e+06,676334.399319
20221,267125,11/01/19,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1.781137e+06,632448.551144
20223,267130,11/01/19,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1.770293e+06,674936.303809


In [10]:
rrmissed.to_csv('rrmissed.csv', index = False)

In [11]:
def normalize_address(s: str) -> str:
    if pd.isna(s):
        return s
    # Only keep everything before the first comma
    s = s.split(",")[0]
    s = s.upper()
    s = re.sub(r"[.,]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    # (keep the direction/street type replacements here)
    return s

In [12]:
rrmissed["address_norm"] = rrmissed["Incident Address"].apply(normalize_address)

C:\Users\user\AppData\Local\Temp\ipykernel_13756\1901410141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rrmissed["address_norm"] = rrmissed["Incident Address"].apply(normalize_address)


In [13]:
rrmissed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,address_norm
0,25270,11/01/17,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089,3817 CROUCH DR
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565,4028 CLARKSVILLE PIKE
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571,6528 THUNDERBIRD DR
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066,604 CROLEY DR
8,25330,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1.794534e+06,618749.342732,4484 LAVERGNE COUCHVILLE PIKE
...,...,...,...,...,...,...,...,...,...,...,...,...
20218,267110,11/01/19,Trash - Backdoor,Friday/paid backdoor trash,3105 fisher dr,37214.0,RED RIVER,1503,15.0,1.767923e+06,673264.801310,3105 FISHER DR
20220,267121,11/01/19,Trash - Curbside/Alley Missed Pickup,missed,"2709 Crestdale Dr, Nashville, TN 37214, United...",37214.0,RED RIVER,1502,15.0,1.770240e+06,676334.399319,2709 CRESTDALE DR
20221,267125,11/01/19,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1.781137e+06,632448.551144,2731 MURFREESBORO PIKE
20223,267130,11/01/19,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1.770293e+06,674936.303809,2943 WINDEMERE CIR


In [14]:
rrmissed["Date Opened"] = pd.to_datetime(rrmissed["Date Opened"], errors="coerce")
rrmissed["date_only"] = rrmissed["Date Opened"].dt.date

C:\Users\user\AppData\Local\Temp\ipykernel_13756\109242868.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  rrmissed["Date Opened"] = pd.to_datetime(rrmissed["Date Opened"], errors="coerce")
C:\Users\user\AppData\Local\Temp\ipykernel_13756\109242868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rrmissed["Date Opened"] = pd.to_datetime(rrmissed["Date Opened"], errors="coerce")
C:\Users\user\AppData\Local\Temp\ipykernel_13756\109242868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [15]:
rrmissed_dedup = (
    rrmissed.sort_values(['address_norm', 'date_only', 'Date Opened'])
    .drop_duplicates(subset=['address_norm', 'date_only'], keep='first')
    .reset_index(drop=True)
)

In [16]:
rrmissed_dedup

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,address_norm,date_only
0,155122,2019-03-15,Trash - Curbside/Alley Missed Pickup,MISS,"100 Bluefield Square, Nashville, TN 37214, Uni...",37214.0,RED RIVER,1505,15.0,1.770431e+06,666861.601362,100 BLUEFIELD SQUARE,2019-03-15
1,52252,2018-03-07,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Braxton Hill Ct,37204.0,RED RIVER,3302S,25.0,1.733781e+06,640909.303557,100 BRAXTON HILL CT,2018-03-07
2,121431,2018-12-05,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Brook Hollow Rd,37205.0,RED RIVER,1303,23.0,1.708043e+06,642454.642918,100 BROOK HOLLOW RD,2018-12-05
3,30424,2017-11-28,Trash - Curbside/Alley Missed Pickup,missed pick up,100 cedarmont ct,37211.0,RED RIVER,4403,31.0,1.760265e+06,622441.107959,100 CEDARMONT CT,2017-11-28
4,142247,2019-02-14,Trash - Curbside/Alley Missed Pickup,Trash was not collected from any customers on ...,"100 Clydelan Ct, Nashville, TN 37205, United S...",37205.0,RED RIVER,1302,34.0,1.710255e+06,640390.616149,100 CLYDELAN CT,2019-02-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13126,192335,2019-05-31,Trash - Curbside/Alley Missed Pickup,odd # side of street was not picked up.,"Old Tusculum Rd, Antioch, Tennessee, 37013",37013.0,RED RIVER,4404,30.0,1.763332e+06,626852.263728,OLD TUSCULUM RD,2019-05-31
13127,133573,2019-01-23,Trash - Curbside/Alley Missed Pickup,got only one side of street,"Queens Ln, Nashville, TN 37218, United States",37218.0,RED RIVER,3203,1.0,1.714591e+06,686127.550777,QUEENS LN,2019-01-23
13128,25809,2017-11-02,Trash - Curbside/Alley Missed Pickup,Robin Springs Road missed!,Robin Springs Rd,37220.0,RED RIVER,3302S,34.0,1.732450e+06,639784.397978,ROBIN SPRINGS RD,2017-11-02
13129,256189,2019-10-07,Trash - Curbside/Alley Missed Pickup,this street missed,"Trevino Pl, Nashville, TN 37013, United States",37013.0,RED RIVER,3505,29.0,1.785940e+06,636624.255256,TREVINO PL,2019-10-07


In [18]:
rrmissed_dedup.to_csv('rrmissed_dedup.csv', index = False)

In [19]:
rrmissed_dedup["miss_seq"] = rrmissed_dedup.groupby("address_norm").cumcount() + 1
rrmissed_dedup["fine_usd"] = (rrmissed_dedup["miss_seq"] > 1).astype(int) * 200

In [20]:
rrmissed_dedup.to_csv('rrmissed_dedup.csv', index = False)

In [21]:
total_fines = rrmissed_dedup["fine_usd"].sum()
by_address = rrmissed_dedup.groupby("address_norm", as_index=False)["fine_usd"].sum()

In [22]:
total_fines

np.int64(947600)

In [23]:
by_address[by_address['address_norm']=='WESTBORO DR']

,address_norm,fine_usd
8392,WESTBORO DR,0


In [24]:
rrmissed_dedup[rrmissed_dedup['address_norm']== 'WESTBORO DR'].head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,address_norm,date_only,miss_seq,fine_usd
13130,232129,2019-08-15,Trash - Curbside/Alley Missed Pickup,miss,"Westboro Dr, Nashville, TN 37209, United States",37209.0,RED RIVER,4203,20.0,1.710106e+06,662979.800016,WESTBORO DR,2019-08-15,1,0


In [25]:
top10 = by_address.nlargest(10, 'fine_usd')
top10

,address_norm,fine_usd
752,12546 OLD HICKORY BLVD,4000
6326,5135 HICKORY HOLLOW PKWY,3800
4415,3710 N NATCHEZ CT,3600
1149,1584 BELL RD,3400
6992,607 ESTES RD,3000
7923,802 CRESCENT RD,3000
1128,1537 HARDING PL,2800
3693,320 OLD HICKORY BLVD,2800
7122,617 KINSEY BLVD,2800
1024,14881 OLD HICKORY BLVD,2600


In [26]:
gap = rrmissed_dedup.groupby("address_norm")["Date Opened"].diff().dt.days
rrmissed_dedup["new_event"] = (gap.isna()) | (gap > 6)
rrmissed_dedup["event_id"]  = rrmissed_dedup.groupby("address_norm")["new_event"].cumsum()

In [27]:
rrmissed_dedup

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,address_norm,date_only,miss_seq,fine_usd,new_event,event_id
0,155122,2019-03-15,Trash - Curbside/Alley Missed Pickup,MISS,"100 Bluefield Square, Nashville, TN 37214, Uni...",37214.0,RED RIVER,1505,15.0,1.770431e+06,666861.601362,100 BLUEFIELD SQUARE,2019-03-15,1,0,True,1
1,52252,2018-03-07,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Braxton Hill Ct,37204.0,RED RIVER,3302S,25.0,1.733781e+06,640909.303557,100 BRAXTON HILL CT,2018-03-07,1,0,True,1
2,121431,2018-12-05,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Brook Hollow Rd,37205.0,RED RIVER,1303,23.0,1.708043e+06,642454.642918,100 BROOK HOLLOW RD,2018-12-05,1,0,True,1
3,30424,2017-11-28,Trash - Curbside/Alley Missed Pickup,missed pick up,100 cedarmont ct,37211.0,RED RIVER,4403,31.0,1.760265e+06,622441.107959,100 CEDARMONT CT,2017-11-28,1,0,True,1
4,142247,2019-02-14,Trash - Curbside/Alley Missed Pickup,Trash was not collected from any customers on ...,"100 Clydelan Ct, Nashville, TN 37205, United S...",37205.0,RED RIVER,1302,34.0,1.710255e+06,640390.616149,100 CLYDELAN CT,2019-02-14,1,0,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13126,192335,2019-05-31,Trash - Curbside/Alley Missed Pickup,odd # side of street was not picked up.,"Old Tusculum Rd, Antioch, Tennessee, 37013",37013.0,RED RIVER,4404,30.0,1.763332e+06,626852.263728,OLD TUSCULUM RD,2019-05-31,1,0,True,1
13127,133573,2019-01-23,Trash - Curbside/Alley Missed Pickup,got only one side of street,"Queens Ln, Nashville, TN 37218, United States",37218.0,RED RIVER,3203,1.0,1.714591e+06,686127.550777,QUEENS LN,2019-01-23,1,0,True,1
13128,25809,2017-11-02,Trash - Curbside/Alley Missed Pickup,Robin Springs Road missed!,Robin Springs Rd,37220.0,RED RIVER,3302S,34.0,1.732450e+06,639784.397978,ROBIN SPRINGS RD,2017-11-02,1,0,True,1
13129,256189,2019-10-07,Trash - Curbside/Alley Missed Pickup,this street missed,"Trevino Pl, Nashville, TN 37013, United States",37013.0,RED RIVER,3505,29.0,1.785940e+06,636624.255256,TREVINO PL,2019-10-07,1,0,True,1


In [28]:
rrmissed_dedup

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,address_norm,date_only,miss_seq,fine_usd,new_event,event_id
0,155122,2019-03-15,Trash - Curbside/Alley Missed Pickup,MISS,"100 Bluefield Square, Nashville, TN 37214, Uni...",37214.0,RED RIVER,1505,15.0,1.770431e+06,666861.601362,100 BLUEFIELD SQUARE,2019-03-15,1,0,True,1
1,52252,2018-03-07,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Braxton Hill Ct,37204.0,RED RIVER,3302S,25.0,1.733781e+06,640909.303557,100 BRAXTON HILL CT,2018-03-07,1,0,True,1
2,121431,2018-12-05,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Brook Hollow Rd,37205.0,RED RIVER,1303,23.0,1.708043e+06,642454.642918,100 BROOK HOLLOW RD,2018-12-05,1,0,True,1
3,30424,2017-11-28,Trash - Curbside/Alley Missed Pickup,missed pick up,100 cedarmont ct,37211.0,RED RIVER,4403,31.0,1.760265e+06,622441.107959,100 CEDARMONT CT,2017-11-28,1,0,True,1
4,142247,2019-02-14,Trash - Curbside/Alley Missed Pickup,Trash was not collected from any customers on ...,"100 Clydelan Ct, Nashville, TN 37205, United S...",37205.0,RED RIVER,1302,34.0,1.710255e+06,640390.616149,100 CLYDELAN CT,2019-02-14,1,0,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13126,192335,2019-05-31,Trash - Curbside/Alley Missed Pickup,odd # side of street was not picked up.,"Old Tusculum Rd, Antioch, Tennessee, 37013",37013.0,RED RIVER,4404,30.0,1.763332e+06,626852.263728,OLD TUSCULUM RD,2019-05-31,1,0,True,1
13127,133573,2019-01-23,Trash - Curbside/Alley Missed Pickup,got only one side of street,"Queens Ln, Nashville, TN 37218, United States",37218.0,RED RIVER,3203,1.0,1.714591e+06,686127.550777,QUEENS LN,2019-01-23,1,0,True,1
13128,25809,2017-11-02,Trash - Curbside/Alley Missed Pickup,Robin Springs Road missed!,Robin Springs Rd,37220.0,RED RIVER,3302S,34.0,1.732450e+06,639784.397978,ROBIN SPRINGS RD,2017-11-02,1,0,True,1
13129,256189,2019-10-07,Trash - Curbside/Alley Missed Pickup,this street missed,"Trevino Pl, Nashville, TN 37013, United States",37013.0,RED RIVER,3505,29.0,1.785940e+06,636624.255256,TREVINO PL,2019-10-07,1,0,True,1


In [29]:
grp = rrmissed_dedup.groupby(['address_norm', "event_id"])['Date Opened']
rrmissed_dedup["event_date"]     = grp.transform("min")          # earliest date in the episode
# rrmissed_dedup["first_report"]   = grp.transform("min")
# rrmissed_dedup["last_report"]    = grp.transform("max")
# rrmissed_dedup["reports_in_evt"] = grp.transform("size")         # how many rows folded into this event


In [30]:
rrmissed_dedup["event_seq"] = rrmissed_dedup["event_id"].astype("Int64")

In [31]:
rrmissed_dedup["fine_event"] = (rrmissed_dedup["event_seq"] > 1).astype(int) * 200

In [32]:
rrmissed_dedup["first_event"] = (rrmissed_dedup['Date Opened'] == rrmissed_dedup["event_date"])
rrmissed_dedup["fine_event"] = rrmissed_dedup["fine_event"].where(rrmissed_dedup["first_event"], 0)


In [33]:
rrmissed_dedup.to_csv('rrmissed_dedup.csv', index = False)

In [34]:
total_fines = int(rrmissed_dedup["fine_event"].sum())
unique_events = int(rrmissed_dedup.drop_duplicates(['address_norm', "event_id"]).shape[0])

In [35]:
total_fines

827400

In [36]:
unique_events

12530

In [37]:
rrmissed_dedup['Zip Code'] = rrmissed_dedup['Zip Code'].apply(lambda x: str(int(x)).zfill(5) if pd.notnull(x) and np.isfinite(x) else None)

In [39]:
rrmissed_dedup

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,...,address_norm,date_only,miss_seq,fine_usd,new_event,event_id,event_date,event_seq,fine_event,first_event
0,155122,2019-03-15,Trash - Curbside/Alley Missed Pickup,MISS,"100 Bluefield Square, Nashville, TN 37214, Uni...",37214,RED RIVER,1505,15.0,1.770431e+06,...,100 BLUEFIELD SQUARE,2019-03-15,1,0,True,1,2019-03-15,1,0,True
1,52252,2018-03-07,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Braxton Hill Ct,37204,RED RIVER,3302S,25.0,1.733781e+06,...,100 BRAXTON HILL CT,2018-03-07,1,0,True,1,2018-03-07,1,0,True
2,121431,2018-12-05,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Brook Hollow Rd,37205,RED RIVER,1303,23.0,1.708043e+06,...,100 BROOK HOLLOW RD,2018-12-05,1,0,True,1,2018-12-05,1,0,True
3,30424,2017-11-28,Trash - Curbside/Alley Missed Pickup,missed pick up,100 cedarmont ct,37211,RED RIVER,4403,31.0,1.760265e+06,...,100 CEDARMONT CT,2017-11-28,1,0,True,1,2017-11-28,1,0,True
4,142247,2019-02-14,Trash - Curbside/Alley Missed Pickup,Trash was not collected from any customers on ...,"100 Clydelan Ct, Nashville, TN 37205, United S...",37205,RED RIVER,1302,34.0,1.710255e+06,...,100 CLYDELAN CT,2019-02-14,1,0,True,1,2019-02-14,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13126,192335,2019-05-31,Trash - Curbside/Alley Missed Pickup,odd # side of street was not picked up.,"Old Tusculum Rd, Antioch, Tennessee, 37013",37013,RED RIVER,4404,30.0,1.763332e+06,...,OLD TUSCULUM RD,2019-05-31,1,0,True,1,2019-05-31,1,0,True
13127,133573,2019-01-23,Trash - Curbside/Alley Missed Pickup,got only one side of street,"Queens Ln, Nashville, TN 37218, United States",37218,RED RIVER,3203,1.0,1.714591e+06,...,QUEENS LN,2019-01-23,1,0,True,1,2019-01-23,1,0,True
13128,25809,2017-11-02,Trash - Curbside/Alley Missed Pickup,Robin Springs Road missed!,Robin Springs Rd,37220,RED RIVER,3302S,34.0,1.732450e+06,...,ROBIN SPRINGS RD,2017-11-02,1,0,True,1,2017-11-02,1,0,True
13129,256189,2019-10-07,Trash - Curbside/Alley Missed Pickup,this street missed,"Trevino Pl, Nashville, TN 37013, United States",37013,RED RIVER,3505,29.0,1.785940e+06,...,TREVINO PL,2019-10-07,1,0,True,1,2019-10-07,1,0,True


In [40]:
trash['Zip Code'] = trash['Zip Code'].astype('Int64').astype(str).str.zfill(5)

In [41]:
rrmissed_dedup

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,...,address_norm,date_only,miss_seq,fine_usd,new_event,event_id,event_date,event_seq,fine_event,first_event
0,155122,2019-03-15,Trash - Curbside/Alley Missed Pickup,MISS,"100 Bluefield Square, Nashville, TN 37214, Uni...",37214,RED RIVER,1505,15.0,1.770431e+06,...,100 BLUEFIELD SQUARE,2019-03-15,1,0,True,1,2019-03-15,1,0,True
1,52252,2018-03-07,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Braxton Hill Ct,37204,RED RIVER,3302S,25.0,1.733781e+06,...,100 BRAXTON HILL CT,2018-03-07,1,0,True,1,2018-03-07,1,0,True
2,121431,2018-12-05,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Brook Hollow Rd,37205,RED RIVER,1303,23.0,1.708043e+06,...,100 BROOK HOLLOW RD,2018-12-05,1,0,True,1,2018-12-05,1,0,True
3,30424,2017-11-28,Trash - Curbside/Alley Missed Pickup,missed pick up,100 cedarmont ct,37211,RED RIVER,4403,31.0,1.760265e+06,...,100 CEDARMONT CT,2017-11-28,1,0,True,1,2017-11-28,1,0,True
4,142247,2019-02-14,Trash - Curbside/Alley Missed Pickup,Trash was not collected from any customers on ...,"100 Clydelan Ct, Nashville, TN 37205, United S...",37205,RED RIVER,1302,34.0,1.710255e+06,...,100 CLYDELAN CT,2019-02-14,1,0,True,1,2019-02-14,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13126,192335,2019-05-31,Trash - Curbside/Alley Missed Pickup,odd # side of street was not picked up.,"Old Tusculum Rd, Antioch, Tennessee, 37013",37013,RED RIVER,4404,30.0,1.763332e+06,...,OLD TUSCULUM RD,2019-05-31,1,0,True,1,2019-05-31,1,0,True
13127,133573,2019-01-23,Trash - Curbside/Alley Missed Pickup,got only one side of street,"Queens Ln, Nashville, TN 37218, United States",37218,RED RIVER,3203,1.0,1.714591e+06,...,QUEENS LN,2019-01-23,1,0,True,1,2019-01-23,1,0,True
13128,25809,2017-11-02,Trash - Curbside/Alley Missed Pickup,Robin Springs Road missed!,Robin Springs Rd,37220,RED RIVER,3302S,34.0,1.732450e+06,...,ROBIN SPRINGS RD,2017-11-02,1,0,True,1,2017-11-02,1,0,True
13129,256189,2019-10-07,Trash - Curbside/Alley Missed Pickup,this street missed,"Trevino Pl, Nashville, TN 37013, United States",37013,RED RIVER,3505,29.0,1.785940e+06,...,TREVINO PL,2019-10-07,1,0,True,1,2019-10-07,1,0,True


In [42]:
rrmissed_dedup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13131 entries, 0 to 13130
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Request Number    13131 non-null  int64         
 1   Date Opened       13131 non-null  datetime64[ns]
 2   Request           13131 non-null  object        
 3   Description       13113 non-null  object        
 4   Incident Address  13131 non-null  object        
 5   Zip Code          13099 non-null  object        
 6   Trash Hauler      13131 non-null  object        
 7   Trash Route       13129 non-null  object        
 8   Council District  13116 non-null  float64       
 9   State Plan X      13126 non-null  float64       
 10  State Plan Y      13126 non-null  float64       
 11  address_norm      13131 non-null  object        
 12  date_only         13131 non-null  object        
 13  miss_seq          13131 non-null  int64         
 14  fine_usd          1313

In [43]:
rrmissed_dedup = rrmissed_dedup[rrmissed_dedup['Request'] != 'Damage to Property']

In [44]:
rrmissed_dedup

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,...,address_norm,date_only,miss_seq,fine_usd,new_event,event_id,event_date,event_seq,fine_event,first_event
0,155122,2019-03-15,Trash - Curbside/Alley Missed Pickup,MISS,"100 Bluefield Square, Nashville, TN 37214, Uni...",37214,RED RIVER,1505,15.0,1.770431e+06,...,100 BLUEFIELD SQUARE,2019-03-15,1,0,True,1,2019-03-15,1,0,True
1,52252,2018-03-07,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Braxton Hill Ct,37204,RED RIVER,3302S,25.0,1.733781e+06,...,100 BRAXTON HILL CT,2018-03-07,1,0,True,1,2018-03-07,1,0,True
2,121431,2018-12-05,Trash - Curbside/Alley Missed Pickup,Missed- trash,100 Brook Hollow Rd,37205,RED RIVER,1303,23.0,1.708043e+06,...,100 BROOK HOLLOW RD,2018-12-05,1,0,True,1,2018-12-05,1,0,True
3,30424,2017-11-28,Trash - Curbside/Alley Missed Pickup,missed pick up,100 cedarmont ct,37211,RED RIVER,4403,31.0,1.760265e+06,...,100 CEDARMONT CT,2017-11-28,1,0,True,1,2017-11-28,1,0,True
4,142247,2019-02-14,Trash - Curbside/Alley Missed Pickup,Trash was not collected from any customers on ...,"100 Clydelan Ct, Nashville, TN 37205, United S...",37205,RED RIVER,1302,34.0,1.710255e+06,...,100 CLYDELAN CT,2019-02-14,1,0,True,1,2019-02-14,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13126,192335,2019-05-31,Trash - Curbside/Alley Missed Pickup,odd # side of street was not picked up.,"Old Tusculum Rd, Antioch, Tennessee, 37013",37013,RED RIVER,4404,30.0,1.763332e+06,...,OLD TUSCULUM RD,2019-05-31,1,0,True,1,2019-05-31,1,0,True
13127,133573,2019-01-23,Trash - Curbside/Alley Missed Pickup,got only one side of street,"Queens Ln, Nashville, TN 37218, United States",37218,RED RIVER,3203,1.0,1.714591e+06,...,QUEENS LN,2019-01-23,1,0,True,1,2019-01-23,1,0,True
13128,25809,2017-11-02,Trash - Curbside/Alley Missed Pickup,Robin Springs Road missed!,Robin Springs Rd,37220,RED RIVER,3302S,34.0,1.732450e+06,...,ROBIN SPRINGS RD,2017-11-02,1,0,True,1,2017-11-02,1,0,True
13129,256189,2019-10-07,Trash - Curbside/Alley Missed Pickup,this street missed,"Trevino Pl, Nashville, TN 37013, United States",37013,RED RIVER,3505,29.0,1.785940e+06,...,TREVINO PL,2019-10-07,1,0,True,1,2019-10-07,1,0,True
